<a href="https://colab.research.google.com/github/bsmingam/TrabajoDeTitulacion/blob/main/Script%20principal%20del%20%20trabajo%20de%20titulacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importar las librerias y/o paquetes necesarios para el proceso de ML**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime
import unicodedata
import numpy as np
import collections
from sklearn import linear_model
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from difflib import SequenceMatcher as SM
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler 

# New section

# New section

**Autentificación para poder acceder a los archivos almacenados en Google Drive**

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

**Abrir el csv y se almacenarlo en un dataframe**

In [ ]:
df = pd.DataFrame()
def _loadCsv():
  global df
  id = '17_JBT4y9XoJYOAqui_tKxvICsbMOckMX'
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('dataset.csv')
  df = pd.read_csv('dataset.csv', converters={'cedula': lambda x: str(x), 'asignatura': lambda x: str(x.strip())})
_loadCsv()
df.shape

(36362, 51)

**Eliminamos registros que tengan valores nulos en la variables de interes**

In [ ]:
_loadCsv()
dataset = df.copy(deep=True)

dataset.dropna(subset=['asignatura'], inplace=True)
dataset.dropna(subset=['estado_asignatura'], inplace=True)
dataset.dropna(subset=['estado_matricula'], inplace=True)
dataset.dropna(subset=['cedula'], inplace=True)

dataset.drop(dataset[dataset['estado_matricula']=="Matriculada"].index, inplace=True)
dataset.drop(dataset[dataset['malla_curricular']=="Exámen complexivo"].index, inplace=True)
dataset.drop(dataset[dataset['malla_curricular']=="UTE"].index, inplace=True)
dataset.drop(dataset[dataset['estado_matricula']=="Reubicada"].index, inplace=True)

dataset.shape

(30187, 51)

**Eliminar las columnas que no se vinculan con los objetivos del negocio, eliminamos los diacríticos y convertimos a mayúsculas los registros**

In [ ]:
columns=['facultad', 'nacionalidad', 'modalidad','pais_nacimiento', 'provincia_nacimiento', 'tipo_colegio', 'ciudad_nacimiento', 'direccion_nacimiento', 'pais_actual', 'provincia_actual', 'canton_actual', 
         'parroquia_actual', 'ciudad_actual','direccion_actual', 'colegio', 'pais_colegio', 'provincia_colegio', 'canton_colegio', 'numero_matricula', 'jornada','obligatoria', 'arrastrable',
         'asistencia_obligatoria', 'nota_ingresada', 'nota_ponderada', 'nombres', 'apellidos', 'promedio_matricula', 'porcentaje_asistencias', 'homologada', 'observacion_homologacion'
        ]
dataset = dataset.drop(columns, axis=1)
cols = dataset.select_dtypes(include=[np.object]).columns 
dataset[cols] = dataset[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper())

**Creamos los métodos que se utilizarán mas adelante**

In [ ]:
def validarCedulaEcuador(cedula):
  d_verificador = int(cedula[9]);
  sum = 0
  for i in range(len(cedula)-1):
    char = int(cedula[i])
    if(i % 2 == 0):
      char = char * 2
      if(char > 9):
        char = char - 9
    sum +=char
  result = 10 - (sum % 10)
  if(result == 10):
    result = 0
    d_verificador = 0
  return result == d_verificador

def encontrarCedulaSemejante(cedulas, ced):
  lista = list()
  for cedula in cedulas:
    cedula = str(cedula)
    if((SM(None, cedula, ced).ratio() >= 0.9) and (SM(None, cedula, ced).ratio() != 1.0)):
      if((cedula in lista) == False):
        lista.append(str(cedula))
  return  lista

def encontrarAsignaturaSemejante(asignaturas, asig):
  lista = list()
  for asignatura in asignaturas:
    asignatura = str(asignatura)
    if((SM(None, asignatura, asig).ratio() >= 0.85) and (SM(None, asignatura, asig).ratio() != 1.0)):
      if((asignatura in lista) == False):
        lista.append(asignatura)
  return  lista

def cal_edad(f):
  today = date.today()
  fnac = datetime.strptime(f, '%Y-%m-%d').date()
  return today.year - fnac.year - ((today.month, today.day) < (fnac.month, fnac.day))

def format(cad):
  cad = cad.strip() 
  return cad.replace(" ", "_") 

def isColumn(lista, col):
  for x in lista:
    if x == col:
      return True
  return False

**Sistetizar la información**

In [ ]:
dataset.loc[dataset.paralelo == "A1", 'paralelo'] = "A"
dataset.loc[dataset.paralelo == "A2", 'paralelo'] = "A"
dataset.loc[dataset.paralelo == "A3", 'paralelo'] = "A"
dataset.loc[dataset.paralelo == "A4", 'paralelo'] = "A"
dataset.loc[dataset.paralelo != "A", 'paralelo'] = "O"

dataset.loc[dataset.estado_matricula == "REUBICADA", 'estado_matricula'] = "REPROBADA"
dataset.loc[dataset.etnia != "MESTIZO", 'etnia'] = "OTRO"

dataset.loc[dataset.estado_civil == "SOLTERO(A)", 'estado_civil'] = "SOLTERO"
dataset.loc[dataset.estado_civil != "SOLTERO", 'estado_civil'] = "OTRO"
dataset.loc[dataset.canton_nacimiento != "LOJA", 'canton_nacimiento'] = "OTRO"

dataset.loc[dataset.numero_hijos == 0, 'numero_hijos'] = "NO"
dataset.loc[dataset.numero_hijos != "NO", 'numero_hijos'] = "SI"
dataset.loc[dataset.ingreso_estudiante == 0, 'ingreso_estudiante'] = "NO"
dataset.loc[dataset.ingreso_estudiante != "NO", 'ingreso_estudiante'] = "SI"

**Codificar las etiquetas de destino (Código de categoría)**

In [ ]:
label_encoder = LabelEncoder()
dataset['estado_matricula'] = label_encoder.fit_transform(dataset['estado_matricula'])

**Convertir las variables categóricas en variables ficticias (indicadoras)**

In [ ]:
dummies = pd.get_dummies(dataset['estado_asignatura'])
dataset['estado_asignatura'] = dummies.REPROBADA

dummies = pd.get_dummies(dataset["genero"])
dataset['genero'] = dummies.FEMENINO

dummies = pd.get_dummies(dataset['estado_civil'])
dataset['estado_civil'] = dummies.OTRO

dummies = pd.get_dummies(dataset['etnia'])
dataset['etnia'] = dummies.OTRO

dummies = pd.get_dummies(dataset['sector_procedencia'])
dataset['sector_procedencia'] = dummies.RURAL

dummies = pd.get_dummies(dataset['canton_nacimiento'])
dataset['canton_nacimiento'] = dummies.OTRO

dummies = pd.get_dummies(dataset['trabaja'])
dataset['trabaja'] = dummies.SI

dummies = pd.get_dummies(dataset['numero_hijos'])
dataset['numero_hijos'] = dummies.SI

dummies = pd.get_dummies(dataset['ingreso_estudiante'])
dataset['ingreso_estudiante'] = dummies.SI

dummies = pd.get_dummies(dataset["paralelo"])
dataset['paralelo'] = dummies.A

**Generar el nuevo campo edad y aplicar el formato a asiganturas**

In [ ]:
dataset['edad'] = dataset.apply(lambda x: cal_edad(x['fecha_nacimiento']), axis=1)
dataset['asignatura'] = dataset.apply(lambda x: format(x['asignatura']), axis=1)

**Eliminar nombres redundantes de las asignaturas**

In [ ]:
dataset.loc[dataset.asignatura == "ECUACIONES_DIFERENCIAL", 'asignatura'] = "ECUACIONES_DIFERENCIALES"
dataset.loc[dataset.asignatura == "CONTROL_AUTOMATICO_Y_ASISTIDO_POR_COMPUTADORES", 'asignatura'] = "CONTROL_AUTOMATIZADO_ASISTIDO_POR_COMPUTADORES"

dataset.loc[dataset.asignatura == "FISICA_I", 'asignatura'] = "FISICA"
dataset.loc[dataset.asignatura == "FISICA_1", 'asignatura'] = "FISICA"
dataset.loc[dataset.asignatura == "ESTRUCTURA_DE_DATOS_II", 'asignatura'] = "ESTRUCTURA_DE_DATOS"
dataset.loc[dataset.asignatura == "ESTRUCTURA_DE_DATOS_OO", 'asignatura'] = "ESTRUCTURA_DE_DATOS"
dataset.loc[dataset.asignatura == "INGENIERIA_DEL_SOFTWARE", 'asignatura'] = "INGENIERIA_DEL_SOFTWARE_I"
dataset.loc[dataset.asignatura == "INGENIERIA_DE_SOFTWARE", 'asignatura'] = "INGENIERIA_DEL_SOFTWARE_I"
dataset.loc[dataset.asignatura == "DISENO_Y_GESTION_DE_BASES_DE_DATOS", 'asignatura'] = "DISENO_Y_GESTION_DE_BASE_DE_DATOS"
dataset.loc[dataset.asignatura == "ARQUITECTURA_DE_COMPUTADORES", 'asignatura'] = "ARQUITECTURA_DE_COMPUTADORAS"
dataset.loc[dataset.asignatura == "INTELIGENCIA_ARIFICIAL", 'asignatura'] = "INTELIGENCIA_ARTIFICIAL"
dataset.loc[dataset.asignatura == "DIENO_DIGITAL", 'asignatura'] = "DISENO_DIGITAL"
dataset.loc[dataset.asignatura == "SIMULACIOM", 'asignatura'] = "SIMULACION"
dataset.loc[dataset.asignatura == "FUNDAMENTOS_BASICOS_DE_COMPUTACION", 'asignatura'] = "FUNDAMENTOS_BASICOS_DE_LA_COMPUTACION"

dataset.loc[dataset.asignatura == "PROYECTOS_INFORMATICOS", 'asignatura'] = "PROYECTOS_INFORMATICOS_I"
dataset.loc[dataset.asignatura == "ANALISI_NUMERICO", 'asignatura'] = "ANALISIS_NUMERICO" 
dataset.loc[dataset.asignatura == "SISTEMAS_DE_INFORMACION_I", 'asignatura'] = "SISTEMAS_DE_INFORMACION"
dataset.loc[dataset.asignatura == "SISTEMAS_DE_INFORMACION_II", 'asignatura'] = "SISTEMAS_DE_INFORMACION"
dataset.loc[dataset.asignatura == "SISTEMAS_DE_INFORMACION_2", 'asignatura'] = "SISTEMAS_DE_INFORMACION"
dataset.loc[dataset.asignatura == "SISTEMAS_DE_INFORMACION_1", 'asignatura'] = "SISTEMAS_DE_INFORMACION"

dataset.loc[dataset.asignatura == "ANTEPROYECTOS_DE_TESIS", 'asignatura'] = "PROYECTO_DE_TRABAJO_DE_TITULACION"
dataset.loc[dataset.asignatura == "ANTEPROYECTOS", 'asignatura'] = "PROYECTO_DE_TRABAJO_DE_TITULACION"

dataset.loc[dataset.asignatura == "LENGUAJES_FORMALES_Y_TEORIA_DE_AUTOMATAS", 'asignatura'] = "AUTOMATAS_Y_LENGUAJES_FORMALES"
dataset.loc[dataset.asignatura == "LENGUAJES_FORMALES", 'asignatura'] = "AUTOMATAS_Y_LENGUAJES_FORMALES"

dataset.loc[dataset.asignatura == "MATEMATICAS", 'asignatura'] = "CALCULO_DIFERENCIAL"
dataset.loc[dataset.asignatura == "MATEMATICAS_2", 'asignatura'] = "CALCULO_INTEGRAL"
dataset.loc[dataset.asignatura == "CALCULO_II", 'asignatura'] = "CALCULO_INTEGRAL"

dataset.loc[dataset.asignatura == "FUNDAMENTOS_BASICOS_DE_LA_COMPUTACION", 'asignatura'] = "PROGRAMACION_I"
dataset.loc[dataset.asignatura == "PROGRAMACION_BASICA", 'asignatura'] = "PROGRAMACION_I"

**Código para identificar las asignaturas semejantes**

In [ ]:
asignaturas = dataset['asignatura']
asignaturas = asignaturas.drop_duplicates().to_list()
print(len(asignaturas))
for asig in asignaturas:
  asig = str(asig)
  print (asig , " ~= " , str(encontrarAsignaturaSemejante(asignaturas, asig)))

87
ELECTRICIDAD  ~=  []
ALGEBRA_LINEAL  ~=  []
COMUNICACION_PROFESIONAL  ~=  []
INTRODUCCION_A_LAS_CIENCIAS_DE_LA_COMPUTACION  ~=  []
TEORIA_DE_LA_PROGRAMACION  ~=  []
MATEMATICAS_DISCRETAS  ~=  []
PROGRAMACION_ORIENTADA_A_OBJETOS  ~=  []
DISENO_DE_CIRCUITOS  ~=  []
TECNOLOGIA_Y_CAMBIO_SOCIAL  ~=  []
CALCULO_DIFERENCIAL  ~=  []
TEORIA_DE_LA_DISTRIBUCION_Y_PROBABILIDAD  ~=  []
ECUACIONES_DIFERENCIALES  ~=  []
CATEDRA_INTEGRADORA:_PROYECTO_DE_VINCULACION:_"CENTRO_DE_ASESORIA_TECNOLOGICA"  ~=  []
ESTRUCTURAS_DE_DATOS_AVANZADAS  ~=  []
ESTADISTICA_ANALITICA  ~=  []
ARQUITECTURA_DE_ORDENADORES  ~=  []
CALCULO_INTEGRAL  ~=  []
BANCOS_DE_DATOS  ~=  []
ESTRUCTURAS_DE_DATOS_Y_ALGORITMOS_FUNDAMENTALES  ~=  []
INGENIERIA_DE_LA_CONTAMINACION  ~=  []
PROCESOS_DE_SOFTWARE  ~=  []
PROCESAMIENTO_DE_TRANSACCIONES  ~=  []
QUIMICA  ~=  []
EXPRESION_ORAL_Y_ESCRITA  ~=  []
FISICA  ~=  []
FUNDAMENTOS_INFORMATICOS  ~=  []
LABORATORIO_DE_FISICA_I  ~=  ['LABORATORIO_DE_FISICA_II']
CALCULO_I  ~=  []
PROGRAMACIO

**Código para identificar los números de cédula que no cumplen con el algoritmo de verificar cédulas ecuatorianas**



In [ ]:
allCedulas = dataset['cedula']
cedulas = allCedulas.drop_duplicates().to_list()
print("Total de cédulas únicas: ", len(cedulas))
lista = list()
for cedula in cedulas:
  cedula = str(cedula)
  if(len(cedula) == 10): 
    if(validarCedulaEcuador(cedula)==False):
      lista.append(cedula)
  else:
    lista.append(cedula)

print("Las cédulas no validadas son:", len(lista))
print(lista)

Total de cédulas únicas:  1270
Las cédulas no validadas son: 2
['77343664', 'FB491880']


**Calculamos la cantidad de registros que tiene cada número de cédula (Cédulas no validadas)**

In [ ]:
def countRowById(dataset, ced):
  filtro = dataset[dataset['cedula'].str.contains(ced, case=False)]['cedula']
  return filtro

for ced in lista:
  print(ced, '\t\t',countRowById(dataset, ced).count())

77343664 		 5
FB491880 		 31


In [ ]:
dataset.shape

(30187, 21)

**Generar dataset´s**

In [ ]:
dataset_sistemas = dataset[dataset['carrera'] == "INGENIERIA EN SISTEMAS"]
dataset_computacion = dataset[dataset['carrera'] == "COMPUTACION"]

In [ ]:
columns = ['carrera', 'periodo_lectivo', 'nivel', 'fecha_nacimiento', 'oferta_academica', 'malla_curricular', 'trabaja'] 
dataset_sistemas = dataset_sistemas.drop(columns, axis=1)
dataset_computacion = dataset_computacion.drop(columns, axis=1)

In [ ]:
print(len(pd.value_counts(dataset['asignatura'])))
print(len(pd.value_counts(dataset_sistemas['asignatura'])))
print(len(pd.value_counts(dataset_computacion['asignatura'])))
print("")
print(len(pd.value_counts(dataset['cedula'])))
print(len(pd.value_counts(dataset_sistemas['cedula'])))
print(len(pd.value_counts(dataset_computacion['cedula'])))

87
71
22

1270
1038
252


**DATASETS POR CICLO**

In [ ]:
dc4 = dataset_computacion[dataset_computacion['ciclo'] == 4]
dc3 = dataset_computacion[dataset_computacion['ciclo'] == 3]
dc2 = dataset_computacion[dataset_computacion['ciclo'] == 2]
dc1 = dataset_computacion[dataset_computacion['ciclo'] == 1]

In [ ]:
ds10 = dataset_sistemas[dataset_sistemas['ciclo'] == 10]
ds9 = dataset_sistemas[dataset_sistemas['ciclo'] == 9]
ds8 = dataset_sistemas[dataset_sistemas['ciclo'] == 8]
ds7 = dataset_sistemas[dataset_sistemas['ciclo'] == 7]
ds6 = dataset_sistemas[dataset_sistemas['ciclo'] == 6]
ds5 = dataset_sistemas[dataset_sistemas['ciclo'] == 5]
ds4 = dataset_sistemas[dataset_sistemas['ciclo'] == 4]
ds3 = dataset_sistemas[dataset_sistemas['ciclo'] == 3]
ds2 = dataset_sistemas[dataset_sistemas['ciclo'] == 2]
ds1 = dataset_sistemas[dataset_sistemas['ciclo'] == 1]

In [ ]:
cedulas_s10 = pd.merge(ds1, ds10, on='cedula')['cedula'].drop_duplicates()
print(cedulas_s10.count())
cedulas_s9 = pd.merge(ds1, ds9, on='cedula')['cedula'].drop_duplicates()
print(cedulas_s9.count())
cedulas_s8 = pd.merge(ds1, ds8, on='cedula')['cedula'].drop_duplicates()
print(cedulas_s8.count())
cedulas_s7 = pd.merge(ds1, ds7, on='cedula')['cedula'].drop_duplicates()
print(cedulas_s7.count())
cedulas_s6 = pd.merge(ds1, ds6, on='cedula')['cedula'].drop_duplicates()
print(cedulas_s6.count())
cedulas_s5 = pd.merge(ds1, ds5, on='cedula')['cedula'].drop_duplicates()
print(cedulas_s5.count())
cedulas_s4 = pd.merge(ds1, ds4, on='cedula')['cedula'].drop_duplicates()
print(cedulas_s4.count())
cedulas_s3 = pd.merge(ds1, ds3, on='cedula')['cedula'].drop_duplicates()
print(cedulas_s3.count())
cedulas_s2 = pd.merge(ds1, ds2, on='cedula')['cedula'].drop_duplicates()
print(cedulas_s2.count())
cedulas_s1 = ds1['cedula'].drop_duplicates()
print(cedulas_s1.count())

165
184
198
221
258
325
359
387
458
571


In [ ]:
ds10 = ds10[ds10['cedula'].isin(cedulas_s10)]
ds9 = ds9[ds9['cedula'].isin(cedulas_s9)]
ds8 = ds8[ds8['cedula'].isin(cedulas_s8)]
ds7 = ds7[ds7['cedula'].isin(cedulas_s7)]
ds6 = ds6[ds6['cedula'].isin(cedulas_s6)]
ds5 = ds5[ds5['cedula'].isin(cedulas_s5)]
ds4 = ds4[ds4['cedula'].isin(cedulas_s4)]
ds3 = ds3[ds3['cedula'].isin(cedulas_s3)]
ds2 = ds2[ds2['cedula'].isin(cedulas_s2)]

In [ ]:
cedulas_c4 = pd.merge(dc1, dc4, on='cedula')['cedula'].drop_duplicates()
print(cedulas_c4.count())
cedulas_c3 = pd.merge(dc1, dc3, on='cedula')['cedula'].drop_duplicates()
print(cedulas_c3.count())
cedulas_c2 = pd.merge(dc1, dc2, on='cedula')['cedula'].drop_duplicates() 
print(cedulas_c2.count())
cedulas_c1 = dc1['cedula'].drop_duplicates() 
print(cedulas_c1.count())

5
20
77
232


In [ ]:
dc4 = dc4[dc4['cedula'].isin(cedulas_c4)]
dc3 = dc3[dc3['cedula'].isin(cedulas_c3)]
dc2 = dc2[dc2['cedula'].isin(cedulas_c2)]

In [ ]:
columnas = ds2.columns.to_list()
columnas.remove('asignatura')
columnas.remove('estado_asignatura')
columnas = ds2['asignatura'].drop_duplicates().to_list() + columnas
dt_tmp = pd.DataFrame(columns=columnas)
dt_tmp.shape

(0, 27)

In [ ]:
dic = {}
ds =  pd.concat([ds1, ds2, ds3], axis=0)
for ced in cedulas_s3:
  aux = ds[ds.cedula==ced]
  dic = {}
  for index, row in aux.iterrows():
    if row['asignatura'] not in dic:
      if isColumn(columnas, row['asignatura']):
        dic[row['asignatura']] = row['estado_asignatura']
    for column_name, value in row.iteritems():
      dic[column_name] = value
  del dic['asignatura']
  del dic['estado_asignatura']
  dt_tmp = dt_tmp.append(dic, ignore_index=True)

print(dt_tmp.shape)

(387, 27)


In [ ]:
dt_tmp.isnull().sum() 

PROGRAMACION_I                             5
CALCULO_INTEGRAL                          64
PROBABILIDAD_E_INFERENCIA_ESTADISTICA    128
ECOLOGIA_Y_MEDIO_AMBIENTE_TECNOLOGICO    130
ALGEBRA_LINEAL                             1
ESTRUCTURA_DE_DATOS                      124
FISICA_II                                 64
GEOMETRIA_ANALITICA                      259
CALCULO_DIFERENCIAL                       10
PROCESO_INVESTIGATIVO                    238
FISICA                                     0
LABORATORIO_DE_FISICA_II                 376
QUIMICA                                   10
EXPRESION_ORAL_Y_ESCRITA                 143
FUNDAMENTOS_INFORMATICOS                 144
cedula                                     0
genero                                     0
estado_civil                               0
etnia                                      0
sector_procedencia                         0
canton_nacimiento                          0
ingreso_estudiante                         0
numero_hij

In [ ]:
dt_tmp = dt_tmp.fillna(method='bfill', limit = 11)

In [ ]:
orden_col = ['cedula', 'ALGEBRA_LINEAL', 'CALCULO_DIFERENCIAL', 'CALCULO_INTEGRAL', 'ECOLOGIA_Y_MEDIO_AMBIENTE_TECNOLOGICO', 'ESTRUCTURA_DE_DATOS', 'EXPRESION_ORAL_Y_ESCRITA', 'FISICA', 'FISICA_II', 'FUNDAMENTOS_INFORMATICOS', 'PROBABILIDAD_E_INFERENCIA_ESTADISTICA', 'PROGRAMACION_I', 'QUIMICA', 'canton_nacimiento', 'ciclo', 'edad', 'estado_civil', 'etnia', 'genero', 'ingreso_estudiante', 'numero_hijos', 'paralelo', 'sector_procedencia', 'estado_matricula']
dt_tmp = dt_tmp[orden_col]

**Selección de caracteristicas**

In [ ]:
features_del = ['cedula', 'ciclo', 'edad', 'estado_civil', 'etnia', 'genero', 'ingreso_estudiante', 'numero_hijos', 'paralelo', 'sector_procedencia']
ONG_data = dt_tmp.drop(features_del, axis=1)
print(ONG_data.shape)
ONG_data.columns

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
#Aplicando el algoritmo univariante de prueba F. 
target = ONG_data['estado_matricula']
k = 5 # número de atributos a seleccionar
entrenar = ONG_data.drop(['estado_matricula'], axis=1)
columnas = list(entrenar.columns.values)
seleccionadas = SelectKBest(f_classif, k = k).fit(entrenar, target)
atrib = seleccionadas.get_support()
atributos = [columnas[i] for i in list(atrib.nonzero()[0])]

(387, 14)


**DIVISIÓN DE LOS DATOS**

In [ ]:
make_classification

features = atributos ###['ALGEBRA_LINEAL', 'CALCULO_DIFERENCIAL',  'FISICA', 'FISICA_II', 'PROGRAMACION_I']

X =  dt_tmp[features].values

y = dt_tmp['estado_matricula'].values
print(pd.value_counts(y))
print(y.shape)
os =  RandomOverSampler(sampling_strategy = 0.45, random_state = 50)

X, y = os.fit_sample(X, y)

print(pd.value_counts(y))

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size = 0.2, random_state = 10)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.2, random_state = 10)

print(y.shape)

0    289
1     98
dtype: int64
(387,)
0    289
1    130
dtype: int64
(419,)


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


**Entrenamiento de los modelos**

In [ ]:
from sklearn import linear_model
linear_model = linear_model.LogisticRegression() 
linear_model.fit(X_train, y_train.ravel())
predict = linear_model.predict(X_validation)
print("LogisticRegression()")
print(linear_model.score(X_test, y_test))
print(metrics.confusion_matrix(y_validation, predict))
print("Accuracy = {0:.3f}".format(metrics.accuracy_score(y_validation, predict)))
print("Precision = {0:.3f}".format(metrics.precision_score(y_validation, predict)))
print("Recall = {0:.3f}".format(metrics.recall_score(y_validation, predict)))

from sklearn.ensemble import RandomForestClassifier
RFC_model = RandomForestClassifier()
RFC_model.fit(X_train, y_train.ravel())
predict = RFC_model.predict(X_validation)
print("\nRandomForestClassifier")
print(RFC_model.score(X_test, y_test))
print(metrics.confusion_matrix(y_validation, predict))
print("Accuracy RF = {0:.3f}".format(metrics.accuracy_score(y_validation, predict)))
print("Precision RF = {0:.3f}".format(metrics.precision_score(y_validation, predict)))
print("Recall RF = {0:.3f}".format(metrics.recall_score(y_validation, predict)))

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier()
clf.fit(X_train, y_train.ravel())
predict = clf.predict(X_validation)
print("\nMLPClassifier")
print(clf.score(X_test, y_test))
print(metrics.confusion_matrix(y_validation, predict))
print("Accuracy = {0:.3f}".format(metrics.accuracy_score(y_validation, predict)))
print("Precision = {0:.3f}".format(metrics.precision_score(y_validation, predict)))
print("Recall = {0:.3f}".format(metrics.recall_score(y_validation, predict)))

LogisticRegression()
0.8059701492537313
[[55  3]
 [10 16]]
Accuracy = 0.845
Precision = 0.842
Recall = 0.615

RandomForestClassifier
0.7910447761194029
[[57  1]
 [11 15]]
Accuracy RF = 0.857
Precision RF = 0.938
Recall RF = 0.577

MLPClassifier
0.8059701492537313
[[57  1]
 [11 15]]
Accuracy = 0.857
Precision = 0.938
Recall = 0.577


**Test**

In [ ]:
predict = linear_model.predict(X_test)
print("LogisticRegression()")
print(metrics.confusion_matrix(y_test, predict))
print("Accuracy = {0:.3f}".format(metrics.accuracy_score(y_test, predict)))
print("Precision = {0:.3f}".format(metrics.precision_score(y_test, predict)))
print("Recall = {0:.3f}".format(metrics.recall_score(y_test, predict)))

predict = RFC_model.predict(X_test)
print("\nRandomForestClassifier")
print(metrics.confusion_matrix(y_test, predict))
print("Accuracy RF = {0:.3f}".format(metrics.accuracy_score(y_test, predict)))
print("Precision RF = {0:.3f}".format(metrics.precision_score(y_test, predict)))
print("Recall RF = {0:.3f}".format(metrics.recall_score(y_test, predict)))

predict = clf.predict(X_test)
print("\nMLPClassifier")
print(metrics.confusion_matrix(y_test, predict))
print("Accuracy = {0:.3f}".format(metrics.accuracy_score(y_test, predict)))
print("Precision = {0:.3f}".format(metrics.precision_score(y_test, predict)))
print("Recall = {0:.3f}".format(metrics.recall_score(y_test, predict)))

LogisticRegression()
[[41  2]
 [11 13]]
Accuracy = 0.806
Precision = 0.867
Recall = 0.542

RandomForestClassifier
[[40  3]
 [11 13]]
Accuracy RF = 0.791
Precision RF = 0.812
Recall RF = 0.542

MLPClassifier
[[41  2]
 [12 12]]
Accuracy = 0.791
Precision = 0.857
Recall = 0.500


In [ ]:
from sklearn.externals import joblib 
joblib.dump(clf, 'modelo_entrenado.pkl')
#clf_load = joblib.load('modelo_entrenado.pkl')

from google.colab import files
#dt_tmp.to_csv('dataset10.csv') 
files.download('modelo_entrenado.pkl')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>